In [7]:
import datetime

import numpy as np
import pandas as pd
from scipy import optimize

import plotly
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from plotly.graph_objs.scatter import Line
from plotly.graph_objs import Scatter

In [19]:
data = np.loadtxt('ex2data2.txt', delimiter=',')
X_orig = data[:, [0, 1]]
y = data[:, 2]
X.shape, y.shape

((118, 28), (118,))

In [20]:
def map_feature(x1, x2):
    degree = 6
    out = np.ones(x1.shape[0])
    for i in range(1, degree + 1):
        for j in range(i + 1):
            col = (x1 ** (i-j)) * (x2 ** j)
            out = np.c_[out, col]
    return out

X = map_feature(X_orig[:, 0], X_orig[:, 1])
X

array([[ 1.00000000e+00,  5.12670000e-02,  6.99560000e-01, ...,
         6.29470940e-04,  8.58939846e-03,  1.17205992e-01],
       [ 1.00000000e+00, -9.27420000e-02,  6.84940000e-01, ...,
         1.89305413e-03, -1.39810280e-02,  1.03255971e-01],
       [ 1.00000000e+00, -2.13710000e-01,  6.92250000e-01, ...,
         1.04882142e-02, -3.39734512e-02,  1.10046893e-01],
       ...,
       [ 1.00000000e+00, -4.84450000e-01,  9.99270000e-01, ...,
         2.34007252e-01, -4.82684337e-01,  9.95627986e-01],
       [ 1.00000000e+00, -6.33640000e-03,  9.99270000e-01, ...,
         4.00328554e-05, -6.31330588e-03,  9.95627986e-01],
       [ 1.00000000e+00,  6.32650000e-01, -3.06120000e-02, ...,
         3.51474517e-07, -1.70067777e-08,  8.22905998e-10]])

In [53]:
def plot(X, y, reg, fig, row, col):
    step = 50
    u = np.linspace(-1, 1.5, step)
    v = np.linspace(-1, 1.5, step)
    z = np.zeros((step, step))
    
    for i in range(step):
        for j in range(step):
            ui = np.array([u[i]])
            vj = np.array([v[j]])
            z[j, i] = reg.predict_proba(map_feature(ui, vj))[0][1]
            
    
    neg = data[data[:, 2] == 0]
    pos = data[data[:, 2] == 1]
    fig.add_trace(Scatter(x=neg[:, 0], y=neg[:, 1], mode='markers'), row=row, col=col)
    fig.add_trace(Scatter(x=pos[:, 0], y=pos[:, 1], mode='markers'), row=row, col=col)
    fig.add_trace(go.Contour(x=u, y=v, z=z, opacity=0.5, ncontours=2), row=row, col=col)

In [54]:
from sklearn.linear_model import LogisticRegression

fig = make_subplots(rows=2, cols=2)

for i, c in enumerate([1e10, 100, 1, 0.01]):
    reg = LogisticRegression(solver="newton-cg", C=c)
    reg.fit(X, y)
    plot(X, y, reg, fig, i//2 + 1, i%2 + 1)

fig.show()